# Project Stage 3: Entity Matching#

The entity we performed our match was on a set of restaurants in New York City. The data for the tables was taken from two different web sources, namely, Tripadvisor and Yelp.

We used the py_entitymatching package to help in this process.

In [2]:
# Import py_entitymatching package
import py_entitymatching as em
import os
import pandas as pd

## Reading in Input Tables##

We read the CSV files and set 'ID' as the key attribute.

Input table A corresponds to data from TripAdvisor, and input table B corresponds to data from Yelp.

In [3]:
#path_A = 'C:\\Users\\bharg\\Documents\\TripAdvisor_Restaurants.csv'
#path_B = 'C:\\Users\\bharg\\Documents\\Yelp_Restaurants.csv'
path_A = 'C:\\Users\\Aribhit\\TripAdvisor_Restaurants.csv'
path_B = 'C:\\Users\\Aribhit\\Yelp_Restaurants.csv'

In [4]:
A = em.read_csv_metadata(path_A, key='Id', encoding = 'cp1252')

Metadata file is not present in the given path; proceeding to read the csv file.


In [5]:
B = em.read_csv_metadata(path_B, key='Id', encoding = 'cp1252')

Metadata file is not present in the given path; proceeding to read the csv file.


# Data Pre-processing#

Since there was a lot of variance in how *Address* attribute was defined, created a new attribute called *Street* that is extracted from *Address* (by considering the part of the string till the state appears).

Also converted every string type attribute to lower case.

In [6]:
A['Street'] = A.apply(lambda row : row['Address'][0:row['Address'].find('New')],axis=1)
A['Name']=A['Name'].str.lower()
A['Street']=A['Street'].str.lower()
A['Address']=A['Address'].str.lower()

In [7]:
def cleaning(row) :
    for string in ['New','Jersey','NY','NJ']:
        index = row['Address'].find(string)
        if index == -1 :
            continue
        return row['Address'][0:index]
    return row['Address']
    
B['Street'] = B.apply(cleaning,axis =1)
B['Name']=B['Name'].str.lower()
B['Street']=B['Street'].str.lower()
B['Address']=B['Address'].str.lower()

Deleting *Phone* attribute in case it might act as an unique ID and game the system

In [8]:
del A['Phone']
del B['Phone']

## Applying the Blocker##

We have used the combination of two blockers: 
One blocks based on *Name* of the restaurant (Jaccard Measure with 3 grams with a constraint 0.3) and *Street* (Jaccard Measure with 3 grams with a constraint 0.3). 
Next blocker is only on the *Street* attribute(Jaccard Measure with 3 grams with a constraint 0.6).

We use these blockers and then combine the results of two different blockers using union for the following reasons.
*Street* (from *Address*) only because it can capture some pairs where names are same but differ by a new word. Ex. (alfa ristorante, alfa). The constraint is higher - 0.6
*Name* only to capture restaurants that have similar names (constraint is 0.3). But added *Street* based rule on top of that to eliminate chain restaurants with multiple branches at different locations (eg: Shake shacks at Manhattan, Shake Shacks at Brooklyn). The constraint is lower threshold in this case compared to the earlier blocker.

First, get all possible features for blocking.

In [9]:
block_f = em.get_features_for_blocking(A, B, validate_inferred_attr_types=False)

First rule-based blocker uses *Name* and *Street* attributes

In [10]:
rb = em.RuleBasedBlocker()
ab = em.AttrEquivalenceBlocker()
rb.add_rule(['Name_Name_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.5'], block_f)
rb.add_rule(['Street_Street_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.3'], block_f)

'_rule_1'

In [11]:
C = rb.block_tables(A, B, l_output_attrs=['Name', 'Street', 'Address','Cuisines','Take Out'], \
                    r_output_attrs=['Name', 'Street', 'Address','Cuisines','Take Out'], show_progress=True)

0% [##############################] 100% | ETA: 00:00:00

Finding pairs with missing value...



Total time elapsed: 00:00:01
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Second rule-based blocker uses only *Street* attribute

In [12]:
rb2 = em.RuleBasedBlocker()
rb2.add_rule(['Street_Street_jac_qgm_3_qgm_3(ltuple, rtuple) < 0.6'], block_f)
E = rb2.block_tables(A, B, l_output_attrs=['Name', 'Street', 'Address','Cuisines','Take Out'], \
                     r_output_attrs=['Name', 'Street', 'Address','Cuisines','Take Out'], n_jobs=-1,show_progress=True)

Finding pairs with missing value...


Combining blocker1 and blocker2 results to get candidate set C (which is named F in our code).

In [13]:
F = em.combine_blocker_outputs_via_union([C, E])

Running debugger to see if F is good. 41/50 outputs of debugger are bad matches.Therefore we are proceeding with the above 
blocker

In [14]:
dbg = em.debug_blocker(F, A, B, output_size=50)
dbg.head()

,_id,ltable_Id,rtable_Id,ltable_Name,ltable_Address,ltable_Cuisines,ltable_Saturday Opening time,ltable_Saturday Closing time,ltable_Sunday Opening time,ltable_Sunday Closing time,...,ltable_Street,rtable_Name,rtable_Address,rtable_Cuisines,rtable_Saturday Opening time,rtable_Saturday Closing time,rtable_Sunday Opening time,rtable_Sunday Closing time,rtable_Take Out,rtable_Street
0,0,1308,1183,razza pizza artigianale,"275 grove st jersey city, nj 07302-3601",Italian;Pizza;Vegetarian Friendly,05:30 PM,10:30 PM,NaN,NaN,...,"275 grove st jersey city, nj 07302-360",razza pizza artigianale,275 grove stjersey city nj 07302,Italian;Salad,05:30 PM,10:00 PM,CLOSED,CLOSED,Yes,275 grove st
1,1,2076,2436,sandro's,"306 e 81st st new york city, ny 10028-3984",Italian,05:00 PM,11:00 PM,04:30 PM,10:00 PM,...,306 e 81st st,antonucci cafe,170 e 81st stnew york ny 10028,Italian,12:00 PM,11:00 PM,12:00 PM,10:00 PM,Yes,170 e 81st st
2,2,1974,3094,strip house,"13 e 12th st new york city, ny 10003-4404",American;Steakhouse;Gluten Free Options,05:00 PM,11:30 PM,05:00 PM,10:00 PM,...,13 e 12th st,strip house speakeasy,11 e 12th stnew york ny 10003,Steakhouses,05:00 PM,11:30 PM,CLOSED,CLOSED,No,11 e 12th st
3,3,3174,2890,porter house,"10 columbus cir ste 4 new york city, ny 10019-1210",American;Steakhouse;Grill,11:30 AM,11:00 PM,11:00 AM,10:00 PM,...,10 columbus cir ste 4,porter house bar and grill,10 columbus cirnew york ny 10019,Steakhouses;American (New);Cocktail Bars,11:30 AM,11:00 PM,11:30 AM,10:00 PM,No,10 columbus cir
4,4,2322,2861,salon de ning rooftop,"700 5th avenue at 55th street new york city, ny 10019",American;Bar,05:00 PM,01:00 AM,05:00 PM,12:00 AM,...,700 5th avenue at 55th street,salon de ning,700 5th avethe peninsula hotelnew york ny 10019,Lounges;Cocktail Bars,05:00 PM,01:00 AM,04:00 PM,12:00 AM,No,700 5th avethe peninsula hotel


In [15]:
F.to_csv("F.csv",index=False)

Taking a sample of 600 tuples from the output, and then we label this sample manually.

In [16]:
S = em.sample_table(F, 600)
S.to_csv('Sample.csv',encoding = 'cp1252')

## Reading the Labelled Sample##
Loading the labeled data table, which is present in a file called 'Labelled_Sample_v2.csv'

In [17]:
L = em.read_csv_metadata("Labelled_Sample_v2.csv", key='_id', encoding = 'cp1252',\
                         ltable=A, rtable=B,fk_ltable='ltable_Id', fk_rtable='rtable_Id')

Metadata file is not present in the given path; proceeding to read the csv file.


Deleting *Phone* attribute again, because it can help determine matches trivially.

In [18]:
del L['ltable_Phone']
del L['rtable_Phone']

## Splitting the Labelled Set##

Splitting the labelled set into training and test set, by putting half the tuple pairs in each.<br>
The development set is called I<br>
The evaluation set is called J

In [19]:
IJ = em.split_train_test(L, train_proportion=0.5, random_state=0)
I = IJ['train']
J = IJ['test']
I.to_csv('I.csv',encoding = 'cp1252')
J.to_csv('J.csv',encoding = 'cp1252')

## Creating ML-matchers##

Initiating 6 different classifiers (Decision Tree, Random Forest, SVM, Naive Bayes, Logistic Regression, Linear Regression) and then, cross validating them on I set.

In [20]:
dt = em.DTMatcher(name='DecisionTree', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
nb = em.NBMatcher(name ='NaiveBayes')
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')

## Selecting Best Matcher ##

First, we obtain all the features we could use for matching. Ft is our feature table

In [21]:
Ft = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

Use the system to generate feature vectors from set I. This is called set H

In [22]:
H = em.extract_feature_vecs(I, 
                            feature_table=Ft, 
                            attrs_after='label',
                            show_progress=False)

Perform matches and display results below (after performing cross-validation)

In [23]:
H = em.impute_table(H, 
                exclude_attrs=['_id', 'ltable_Id', 'rtable_Id', 'label'],
                strategy='mean')

In [24]:
result = em.select_matcher([dt, rf, svm, ln, lg,nb], table=H, 
        exclude_attrs=['_id', 'ltable_Id', 'rtable_Id', 'label'],
        k=5,
        target_attr='label', metric_to_select_matcher='f1', random_state=0)
result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.966252,0.963852,0.964790
1,RF,0.980505,0.973049,0.976478
2,SVM,0.817495,0.986258,0.892202
3,LinReg,0.973183,0.938203,0.955157
4,LogReg,0.966219,0.964915,0.965403
5,NaiveBayes,0.974935,0.964915,0.969759


Picking Random Forest as it has the highest average F1 score. We are not adding any rule based matchers as the precision,recall and F1 scores are already above the required thresholds.

## Evaluating Best Matcher##

As we picked Random Forest as the best matcher, now we apply it on the evaluation set (set J; defined earlier) to find how well it performs.

Create a new Random Forest matcher and train it on set H (feature table obtained from set I):

In [25]:
rf = em.RFMatcher(name='RF', random_state=0)

In [26]:
rf.fit(table=H, 
       exclude_attrs=['_id', 'ltable_Id', 'rtable_Id', 'label'], 
       target_attr='label')

Extracting features from set J:

In [27]:
Test_Ft = em.extract_feature_vecs(J, feature_table=Ft,
                            attrs_after='label', show_progress=False)

In [28]:
Test_Ft = Test_Ft.dropna(axis =0,how ='any')
Test_Ft.to_csv("Test_Ft.csv",index=False)

In [29]:
Test_Ft = em.read_csv_metadata("Test_Ft.csv", key='_id', encoding = 'cp1252',\
                         ltable=A, rtable=B,fk_ltable='ltable_Id', fk_rtable='rtable_Id')

Metadata file is not present in the given path; proceeding to read the csv file.


Computing predictions on set J:

In [30]:
predictions = rf.predict(table=Test_Ft, exclude_attrs=['_id', 'ltable_Id', 'rtable_Id', 'label'], 
              append=True, target_attr='predicted', inplace=False, return_probs=True,
                        probs_attr='proba')

In [31]:
predictions[['_id', 'ltable_Id', 'rtable_Id', 'predicted', 'proba']].head()

,_id,ltable_Id,rtable_Id,predicted,proba
0,1164,2882,3032,0,0.1
1,603,1522,599,1,1.0
2,1290,3111,1845,1,1.0
3,540,1350,1949,0,0.0
4,1554,3724,1508,1,1.0


In [32]:
eval_result = em.eval_matches(predictions, 'label', 'predicted')
em.print_eval_summary(eval_result)

Precision : 97.37% (148/152)
Recall : 97.37% (148/152)
F1 : 97.37%
False positives : 4 (out of 152 positive predictions)
False negatives : 4 (out of 67 negative predictions)


## Evaluating Other Matchers ##

Now, we evaluate the performance of other matchers on set J:

In [33]:
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name ='NaiveBayes')

In [34]:
dt.fit(table=H, 
       exclude_attrs=['_id', 'ltable_Id', 'rtable_Id', 'label'], 
       target_attr='label')
svm.fit(table=H, 
       exclude_attrs=['_id', 'ltable_Id', 'rtable_Id', 'label'], 
       target_attr='label')
lg.fit(table=H, 
       exclude_attrs=['_id', 'ltable_Id', 'rtable_Id', 'label'], 
       target_attr='label')
ln.fit(table=H, 
       exclude_attrs=['_id', 'ltable_Id', 'rtable_Id', 'label'], 
       target_attr='label')
nb.fit(table=H, 
       exclude_attrs=['_id', 'ltable_Id', 'rtable_Id', 'label'], 
       target_attr='label')

In [35]:
predictions2 = dt.predict(table=Test_Ft, exclude_attrs=['_id', 'ltable_Id', 'rtable_Id', 'label'], 
              append=True, target_attr='predicted', inplace=False, return_probs=True,
                        probs_attr='proba')
predictions3 = svm.predict(table=Test_Ft, exclude_attrs=['_id', 'ltable_Id', 'rtable_Id', 'label'], 
              append=True, target_attr='predicted', inplace=False)
predictions4 = lg.predict(table=Test_Ft, exclude_attrs=['_id', 'ltable_Id', 'rtable_Id', 'label'], 
              append=True, target_attr='predicted', inplace=False, return_probs=True,
                        probs_attr='proba')
predictions5 = ln.predict(table=Test_Ft, exclude_attrs=['_id', 'ltable_Id', 'rtable_Id', 'label'], 
              append=True, target_attr='predicted', inplace=False)
predictions6 = nb.predict(table=Test_Ft, exclude_attrs=['_id', 'ltable_Id', 'rtable_Id', 'label'], 
              append=True, target_attr='predicted', inplace=False, return_probs=True,
                        probs_attr='proba')

Decision Tree

In [36]:
eval_result2 = em.eval_matches(predictions2, 'label', 'predicted')
em.print_eval_summary(eval_result2)

Precision : 93.59% (146/156)
Recall : 96.05% (146/152)
F1 : 94.81%
False positives : 10 (out of 156 positive predictions)
False negatives : 6 (out of 63 negative predictions)


SVM

In [37]:
eval_result3 = em.eval_matches(predictions3, 'label', 'predicted')
em.print_eval_summary(eval_result3)

Precision : 83.62% (148/177)
Recall : 97.37% (148/152)
F1 : 89.97%
False positives : 29 (out of 177 positive predictions)
False negatives : 4 (out of 42 negative predictions)


Logistic Regression

In [38]:
eval_result4 = em.eval_matches(predictions4, 'label', 'predicted')
em.print_eval_summary(eval_result4)

Precision : 95.39% (145/152)
Recall : 95.39% (145/152)
F1 : 95.39%
False positives : 7 (out of 152 positive predictions)
False negatives : 7 (out of 67 negative predictions)


Linear Regression

In [39]:
eval_result5 = em.eval_matches(predictions5, 'label', 'predicted')
em.print_eval_summary(eval_result5)

Precision : 95.51% (149/156)
Recall : 98.03% (149/152)
F1 : 96.75%
False positives : 7 (out of 156 positive predictions)
False negatives : 3 (out of 63 negative predictions)


Naive Bayes

In [40]:
eval_result6 = em.eval_matches(predictions6, 'label', 'predicted')
em.print_eval_summary(eval_result6)

Precision : 96.73% (148/153)
Recall : 97.37% (148/152)
F1 : 97.05%
False positives : 5 (out of 153 positive predictions)
False negatives : 4 (out of 66 negative predictions)
